In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import sys
from pandas.plotting import scatter_matrix as scatter
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import numpy as np
import nltk
# nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
import sklearn.preprocessing as skp
import sklearn.feature_selection as skfs

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/jacob/nltk_data...


In [2]:
df_us = pd.read_csv("./YouTube Data/USvideos.csv")
df_ca = pd.read_csv("./YouTube Data/CAvideos.csv")
df_uk = pd.read_csv("./YouTube Data/GBvideos.csv")

In [3]:
df_us.head()

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,2kyS6SvSYSE,17.14.11,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,22,2017-11-13T17:13:01.000Z,SHANtell martin,748374,57527,2966,15954,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,False,False,False,SHANTELL'S CHANNEL - https://www.youtube.com/s...
1,1ZAPwfrtAFY,17.14.11,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,24,2017-11-13T07:30:00.000Z,"last week tonight trump presidency|""last week ...",2418783,97185,6146,12703,https://i.ytimg.com/vi/1ZAPwfrtAFY/default.jpg,False,False,False,"One year after the presidential election, John..."
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146033,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...
3,puqaWrEC7tY,17.14.11,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,24,2017-11-13T11:00:04.000Z,"rhett and link|""gmm""|""good mythical morning""|""...",343168,10172,666,2146,https://i.ytimg.com/vi/puqaWrEC7tY/default.jpg,False,False,False,Today we find out if Link is a Nickelback amat...
4,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12T18:01:41.000Z,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095731,132235,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...


In [178]:
df_us.corr(method ='pearson')

,category_id,views,likes,dislikes,comment_count,comments_disabled,ratings_disabled,video_error_or_removed
category_id,1.000000,-0.168231,-0.173921,-0.033547,-0.076307,0.048949,-0.013506,-0.030011
views,-0.168231,1.000000,0.849177,0.472213,0.617621,0.002677,0.015355,-0.002256
likes,-0.173921,0.849177,1.000000,0.447186,0.803057,-0.028918,-0.020888,-0.002641
dislikes,-0.033547,0.472213,0.447186,1.000000,0.700184,-0.004431,-0.008230,-0.001853
comment_count,-0.076307,0.617621,0.803057,0.700184,1.000000,-0.028277,-0.013819,-0.003725
comments_disabled,0.048949,0.002677,-0.028918,-0.004431,-0.028277,1.000000,0.319230,-0.002970
ratings_disabled,-0.013506,0.015355,-0.020888,-0.008230,-0.013819,0.319230,1.000000,-0.001526
video_error_or_removed,-0.030011,-0.002256,-0.002641,-0.001853,-0.003725,-0.002970,-0.001526,1.000000


In [179]:
df_us_drop = df_us.drop(columns = ["description", "title", "tags", "thumbnail_link", "video_id", "publish_time", "trending_date"])
df_us_drop = df_us_drop[0:50]
df_dummy = pd.get_dummies(df_us_drop)

In [180]:
#df_dummy.head()

In [181]:
df_dummy.corr(method ='pearson')

,category_id,views,likes,dislikes,comment_count,comments_disabled,ratings_disabled,video_error_or_removed,channel_title_20th Century Fox,channel_title_Amazon.co.uk,...,channel_title_Tom Scott,channel_title_U2VEVO,channel_title_Vox,channel_title_amc,channel_title_dope2111,channel_title_hp_overload,channel_title_iJustine,channel_title_marshmello,channel_title_nigahiga,channel_title_officer401
category_id,1.000000,-0.150853,-0.163037,-0.219174,-0.169460,0.066129,NaN,NaN,-0.354028,0.066129,...,0.120932,-0.189619,0.084397,0.066129,0.102664,-0.335760,0.139200,-0.189619,0.066129,0.066129
views,-0.150853,1.000000,0.982226,0.934912,0.976258,-0.045667,NaN,NaN,0.001152,-0.045667,...,-0.038737,-0.043647,-0.032182,-0.041068,0.038043,-0.041431,-0.040214,-0.006951,0.075452,-0.043229
likes,-0.163037,0.982226,1.000000,0.927843,0.986101,-0.043032,NaN,NaN,-0.038688,-0.043032,...,-0.028266,-0.036346,-0.027129,-0.041184,-0.000676,-0.038064,-0.030797,0.101686,0.124582,-0.038387
dislikes,-0.219174,0.934912,0.927843,1.000000,0.947234,-0.042734,NaN,NaN,-0.041617,-0.042734,...,-0.042287,-0.041907,-0.013814,-0.037504,-0.007176,-0.033213,-0.032856,-0.014485,0.000177,-0.042399
comment_count,-0.169460,0.976258,0.986101,0.947234,1.000000,-0.040681,NaN,NaN,-0.037936,-0.040681,...,-0.032496,-0.037008,-0.021566,-0.030356,-0.023245,-0.036758,-0.029105,0.026892,0.100729,-0.035628
comments_disabled,0.066129,-0.045667,-0.043032,-0.042734,-0.040681,1.000000,NaN,NaN,-0.020408,1.000000,...,-0.020408,-0.020408,-0.020408,-0.020408,-0.020408,-0.020408,-0.020408,-0.020408,-0.020408,-0.020408
ratings_disabled,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
video_error_or_removed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
channel_title_20th Century Fox,-0.354028,0.001152,-0.038688,-0.041617,-0.037936,-0.020408,NaN,NaN,1.000000,-0.020408,...,-0.020408,-0.020408,-0.020408,-0.020408,-0.020408,-0.020408,-0.020408,-0.020408,-0.020408,-0.020408
channel_title_Amazon.co.uk,0.066129,-0.045667,-0.043032,-0.042734,-0.040681,1.000000,NaN,NaN,-0.020408,1.000000,...,-0.020408,-0.020408,-0.020408,-0.020408,-0.020408,-0.020408,-0.020408,-0.020408,-0.020408,-0.020408


In [19]:
count_vectorizer = CountVectorizer(stop_words='english')
counts = count_vectorizer.fit_transform(df_us['title'].values)

In [13]:
count_vectorizer.vocabulary_

{'want': 9593,
 'talk': 8817,
 'marriage': 5580,
 'trump': 9237,
 'presidency': 6989,
 'week': 9663,
 'tonight': 9088,
 'john': 4751,
 'oliver': 6386,
 'hbo': 4136,
 'racist': 7195,
 'superman': 8703,
 'rudy': 7676,
 'mancuso': 5527,
 'king': 4957,
 'bach': 822,
 'lele': 5198,
 'pons': 6890,
 'nickelback': 6215,
 'lyrics': 5445,
 'real': 7290,
 'fake': 3255,
 'dare': 2364,
 'going': 3829,
 'bald': 853,
 'weeks': 9668,
 'iphone': 4590,
 'roy': 7663,
 'moore': 5979,
 'jeff': 4693,
 'sessions': 7969,
 'cold': 1948,
 'open': 6414,
 'snl': 8288,
 'ice': 4394,
 'cream': 2198,
 'gadgets': 3652,
 'test': 8923,
 'greatest': 3910,
 'showman': 8098,
 'official': 6366,
 'trailer': 9148,
 'hd': 4137,
 '20th': 107,
 'century': 1657,
 'fox': 3551,
 'rise': 7549,
 'robots': 7584,
 'won': 9790,
 'mean': 5697,
 'end': 3040,
 'work': 9808,
 'dion': 2623,
 'lewis': 5232,
 '103': 17,
 'yd': 9881,
 'kick': 4923,
 'return': 7477,
 'td': 8864,
 'vs': 9560,
 'denver': 2502,
 'miss': 5887,
 'play': 6834,
 'nfl'

In [184]:
# print(df_us['category_id'].nunique())
#for i in df_us['category_id'].unique():
#    print(i)
#    print(df_us[df_us['category_id'] == i].count())
#    print("\n")
# df_us[df_us['category_id'] == 1].count()

In [185]:
#df_us.head()

# Mutual Info Regressions

# United States

In [14]:
for i in df_us['category_id'].unique():
    X = count_vectorizer.fit_transform(df_us['title'][df_us['category_id'] == i])
    Y = df_us['views'][df_us['category_id'] == i]
    res = dict(zip(count_vectorizer.get_feature_names(),
               skfs.mutual_info_regression(X, Y, discrete_features=True)
               ))
    print("Category: " + str(i))
    print(sorted(res.items(), key=lambda x: x[1], reverse = True)[0:10])
    print("\n")

Category: 22
[('primitive', 0.030911439814525488), ('technology', 0.030911439814525488), ('grace', 0.028626645514697957), ('helbig', 0.028626645514697957), ('interview', 0.02797910615948651), ('charlamagne', 0.027356605541006718), ('kanye', 0.027356605541006718), ('west', 0.027356605541006718), ('makeup', 0.027147657043132822), ('fan', 0.026940766385219028)]


Category: 24
[('official', 0.06808033882642217), ('trailer', 0.0509552095475132), ('hd', 0.02488706608433411), ('talk', 0.019871366276517533), ('studios', 0.01971914930994867), ('movie', 0.01846344311806436), ('2018', 0.017501520330744702), ('react', 0.016863928526823946), ('hirani', 0.016714144279337817), ('kapoor', 0.016714144279337817)]


Category: 23
[('lele', 0.051676449869815455), ('pons', 0.051676449869815455), ('reading', 0.039423134321247666), ('lip', 0.03861608189443255), ('bad', 0.03607158830646284), ('royal', 0.032957875351119825), ('yiay', 0.03173886845158491), ('wedding', 0.030411665223142204), ('10', 0.025761978305

In [15]:
for i in df_us['category_id'].unique():
    X = count_vectorizer.fit_transform(df_us['tags'][df_us['category_id'] == i])
    Y = df_us['views'][df_us['category_id'] == i]
    res = dict(zip(count_vectorizer.get_feature_names(),
               skfs.mutual_info_regression(X, Y, discrete_features=True)
               ))
    print("Category: " + str(i))
    print(sorted(res.items(), key=lambda x: x[1], reverse = True)[0:10])
    print("\n")

Category: 22
[('safiya', 0.09867605784904643), ('fashion', 0.08550474504227235), ('buzzfeed', 0.07313894925016307), ('interview', 0.064856141404666), ('makeup', 0.064695317347079), ('nygaard', 0.06346433552579223), ('try', 0.06015653125652487), ('family', 0.05858063858788842), ('safia', 0.05667360749811934), ('music', 0.05568963721304332)]


Category: 24
[('movie', 0.05727733589563466), ('trailer', 0.05502965002988791), ('2018', 0.051071889041310436), ('new', 0.047035069525851236), ('official', 0.04224672715009903), ('marvel', 0.0388213239112809), ('black', 0.03760431980878498), ('news', 0.03698091794209213), ('youtube', 0.03646153760917481), ('challenge', 0.035047889947450894)]


Category: 23
[('hannah', 0.0874815128557469), ('funny', 0.08386585402403557), ('anwar', 0.07362884491085286), ('comedy', 0.07260940432970764), ('alesso', 0.07144884130134144), ('inanna', 0.0675793971121077), ('sarkis', 0.0675793971121077), ('shots', 0.06584107606151868), ('sketch', 0.06412069690340183), ('ani

In [25]:
df2 = df_us._get_numeric_data()
df2 = df2.drop(columns='views')

In [26]:
mutual_information = skfs.mutual_info_regression(df2, df_us['views'])

In [27]:
mutual_list_name = []
mutual_list_value = []
count = 0
for i in df2:
    mutual_list_value.append(mutual_information[count])
    mutual_list_name.append(i)
    count += 1


mutual_list = pd.DataFrame(columns = ['Feature', 'Value'])
mutual_list['Feature'] = mutual_list_name
mutual_list['Value'] = mutual_list_value

In [28]:
mutual_list_sorted = mutual_list.sort_values(by='Value', ascending = False)
mutual_list_sorted.reset_index(drop = True)

,Feature,Value
0,likes,1.013962
1,dislikes,0.884180
2,comment_count,0.812184
3,category_id,0.096716
4,comments_disabled,0.005511
5,ratings_disabled,0.003489
6,video_error_or_removed,0.000000


# Canada

In [19]:
for i in df_ca['category_id'].unique():
    X = count_vectorizer.fit_transform(df_ca['title'][df_ca['category_id'] == i])
    Y = df_ca['views'][df_ca['category_id'] == i]
    res = dict(zip(count_vectorizer.get_feature_names(),
               skfs.mutual_info_regression(X, Y, discrete_features=True)
               ))
    print("Category: " + str(i))
    print(sorted(res.items(), key=lambda x: x[1], reverse = True)[0:10])
    print("\n")

Category: 10
[('songs', 0.02579618750346535), ('latest', 0.015713611935045835), ('official', 0.015565748946826163), ('2018', 0.015164273418876606), ('punjabi', 0.014974353149634467), ('video', 0.013194133159135202), ('song', 0.012975946528459126), ('lyrics', 0.010789182722583668), ('launch', 0.010504870694180823), ('enima', 0.009982857131742628)]


Category: 23
[('closer', 0.03676753206399219), ('look', 0.03589249527421812), ('daily', 0.03284499049893319), ('awards', 0.02358429980318877), ('comment', 0.02311157696423649), ('lele', 0.01880226109914862), ('pons', 0.01880226109914862), ('trump', 0.017263496258060673), ('failarmy', 0.016657161120874342), ('rick', 0.015864211834353048)]


Category: 24
[('episode', 0.03288360310588567), ('trailer', 0.030707122113956542), ('official', 0.027540871286543922), ('vaani', 0.018695984588201875), ('rani', 0.018538355389462025), ('chashmah', 0.013723308170714321), ('mehta', 0.013723308170714321), ('ooltah', 0.013723308170714321), ('taarak', 0.0135865

In [20]:
for i in df_ca['category_id'].unique():
    X = count_vectorizer.fit_transform(df_ca['tags'][df_ca['category_id'] == i])
    Y = df_ca['views'][df_ca['category_id'] == i]
    res = dict(zip(count_vectorizer.get_feature_names(),
               skfs.mutual_info_regression(X, Y, discrete_features=True)
               ))
    print("Category: " + str(i))
    print(sorted(res.items(), key=lambda x: x[1], reverse = True)[0:10])
    print("\n")

Category: 10
[('latest', 0.037628930683069495), ('songs', 0.029873021602154104), ('toronto', 0.028866169374094586), ('video', 0.024148370936668773), ('punjabi', 0.02246105942769372), ('oficial', 0.02198947888160785), ('new', 0.021189883966346912), ('2018', 0.020916454737467616), ('single', 0.020522384184819442), ('drake', 0.018725009577536866)]


Category: 23
[('comedy', 0.044588714714204336), ('politics', 0.04069228022091109), ('2017', 0.03951085209801497), ('trump', 0.034923545318072735), ('closer', 0.03331970525346151), ('compilation', 0.03249764079437156), ('funny', 0.0323492077562344), ('weekend', 0.03154580958412456), ('central', 0.031415505441355185), ('satire', 0.0310064757515065)]


Category: 24
[('serial', 0.036987449808473905), ('tv', 0.025809537507446523), ('serials', 0.022820183384605786), ('india', 0.022740277115421703), ('trailer', 0.02245117264567864), ('comedy', 0.022020882035975298), ('2018', 0.020554802089394197), ('radhika', 0.020036505237125968), ('drama', 0.019861

In [30]:
df2 = df_ca._get_numeric_data()
df2 = df2.drop(columns='views')

In [32]:
mutual_information = skfs.mutual_info_regression(df2, df_ca['views'])

In [40]:
mutual_list_name = []
mutual_list_value = []
count = 0
for i in df2:
    mutual_list_value.append(mutual_information[count])
    mutual_list_name.append(i)
    count += 1


mutual_list = pd.DataFrame(columns = ['Feature', 'Value'])
mutual_list['Feature'] = mutual_list_name
mutual_list['Value'] = mutual_list_value

In [41]:
mutual_list_sorted = mutual_list.sort_values(by='Value', ascending = False)
mutual_list_sorted.reset_index(drop = True)

,Feature,Value
0,dislikes,0.795698
1,likes,0.663060
2,comment_count,0.477573
3,category_id,0.100167
4,comments_disabled,0.005031
5,ratings_disabled,0.001891
6,video_error_or_removed,0.000000


# Great Britain

In [22]:
for i in df_uk['category_id'].unique():
    X = count_vectorizer.fit_transform(df_uk['title'][df_uk['category_id'] == i])
    Y = df_uk['views'][df_uk['category_id'] == i]
    res = dict(zip(count_vectorizer.get_feature_names(),
               skfs.mutual_info_regression(X, Y, discrete_features=True)
               ))
    print("Category: " + str(i))
    print(sorted(res.items(), key=lambda x: x[1], reverse = True)[0:10])
    print("\n")

Category: 26
[('alton', 0.06794623739743266), ('brown', 0.06794623739743266), ('reviews', 0.06794623739743266), ('rigorously', 0.06794623739743266), ('vlogmas', 0.06668169647638122), ('makeup', 0.06657154575034374), ('hot', 0.06435298732261474), ('ones', 0.06435298732261474), ('spicy', 0.06435298732261474), ('wings', 0.06435298732261474)]


Category: 24
[('official', 0.08585728668321346), ('marvel', 0.0410948629085055), ('studios', 0.03886098498546242), ('trailer', 0.03662321183442874), ('super', 0.03564260197462421), ('bowl', 0.034935753497737254), ('commercial', 0.034935753497737254), ('teaser', 0.032789425783242176), ('video', 0.03169974741446846), ('war', 0.03070008469154606)]


Category: 10
[('oficial', 0.03234808262282296), ('ft', 0.02036063442325564), ('balvin', 0.018639207448174266), ('bunny', 0.01798485291161178), ('christmas', 0.015238078623784568), ('video', 0.015122240201231207), ('fall', 0.014900460080283562), ('ozuna', 0.014825288501001288), ('live', 0.014473034683545993)

In [23]:
for i in df_uk['category_id'].unique():
    X = count_vectorizer.fit_transform(df_uk['tags'][df_uk['category_id'] == i])
    Y = df_uk['views'][df_uk['category_id'] == i]
    res = dict(zip(count_vectorizer.get_feature_names(),
               skfs.mutual_info_regression(X, Y, discrete_features=True)
               ))
    print("Category: " + str(i))
    print(sorted(res.items(), key=lambda x: x[1], reverse = True)[0:10])
    print("\n")

Category: 26
[('makeup', 0.24979537259267737), ('beauty', 0.1563461632511105), ('recipe', 0.1405481266232107), ('christmas', 0.139136416559698), ('cosmetics', 0.1377184503380231), ('life', 0.13392607378215526), ('fashion', 0.13377538975279246), ('food', 0.1278086797365119), ('tutorial', 0.12631991260878528), ('nikkietutorials', 0.11966353470450897)]


Category: 24
[('trailer', 0.08215167391664502), ('super', 0.07392237263200152), ('movie', 0.0686016023927809), ('official', 0.06709719616434628), ('man', 0.06680031496380989), ('star', 0.06622656494330603), ('marvel', 0.06306252197017348), ('tom', 0.060456031974803315), ('youtube', 0.05872527839770725), ('paul', 0.05863219290323518)]


Category: 10
[('ozuna', 0.03784131868268159), ('latin', 0.03619587168511851), ('video', 0.035446478194550934), ('remix', 0.03511578892796208), ('bunny', 0.03485249087688791), ('oficial', 0.034005069919490705), ('la', 0.02930278813277032), ('cardi', 0.02842040500614984), ('reggaeton', 0.026475718059034592), 

In [42]:
df2 = df_uk._get_numeric_data()
df2 = df2.drop(columns='views')

In [43]:
mutual_information = skfs.mutual_info_regression(df2, df_uk['views'])

In [44]:
mutual_list_name = []
mutual_list_value = []
count = 0
for i in df2:
    mutual_list_value.append(mutual_information[count])
    mutual_list_name.append(i)
    count += 1


mutual_list = pd.DataFrame(columns = ['Feature', 'Value'])
mutual_list['Feature'] = mutual_list_name
mutual_list['Value'] = mutual_list_value

In [45]:
mutual_list_sorted = mutual_list.sort_values(by='Value', ascending = False)
mutual_list_sorted.reset_index(drop = True)

,Feature,Value
0,likes,1.341073
1,dislikes,1.122469
2,comment_count,1.029363
3,category_id,0.172633
4,ratings_disabled,0.006852
5,comments_disabled,0.006784
6,video_error_or_removed,0.001522


# Linear Regressions

In [34]:
#df_us_drop = df_us.drop(columns=['video_id','trending_date','channel_title','publish_time','tags','likes','dislikes',
#                                'comment_count','thumbnail_link','comments_disabled','ratings_disabled',
#                                 'video_error_or_removed','description'])

In [35]:
#X10.shape

In [36]:
#res = dict(zip(count_vectorizer.get_feature_names(),
#               skfs.mutual_info_regression(X10, Y10, discrete_features=True)
#               ))

In [37]:
#sorted(res.items(), key=lambda x: x[1], reverse = True)[0:10]

In [51]:
kbest = skfs.SelectKBest(k=50)
#kbest.VarianceThreshold()
# np.seterr(divide='warn', invalid='warn')
# np.seterr(divide='ignore', invalid='ignore')
kbest.fit(X, Y)
#important = kbest.get_support(True)
# print(important)

/Users/jacob/enter/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:115: RuntimeWarning: divide by zero encountered in true_divide
  f = msb / msw


SelectKBest(k=50, score_func=<function f_classif at 0x1a25bd9a70>)

In [186]:
#kbest = skfs.SelectKBest(k=50)
#kbest.fit(X, Y)
#important = kbest.get_support(True)

In [97]:
#print(clf.fit(X,Y))
#for i in clf.coef_:
#    print(i)

In [99]:
#for c, i in sorted(zip(clf.coef_, important.tolist()), reverse=True):
#     print ("% 20s  | coef: %.2f" % (count_vectorizer.get_feature_names()[i], c))

In [48]:
# print(mutual_information)

In [20]:
df_us['title_channel_tags'] = df_us['title'] + df_us['tags'] + df_us['channel_title']
# X = count_vectorizer.fit_transform(df_us['title_channel_tags'][df_us['category_id'] == 23])
X = count_vectorizer.fit_transform(df_us['title_channel_tags'][df_us['category_id'] == 23])
#item = df_us['category_id'][df_us['category_id'] == 23]
#X = 
Y = df_us['views'][df_us['category_id'] == 23]

In [209]:
X_train, X_test, y_train, y_test = train_test_split(
X, Y, test_size=0.20)

In [215]:
df = pd.DataFrame(X)

In [210]:
clf = LinearRegression()
clf.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [211]:
print(X_test)

  (0, 4117)	1
  (0, 2293)	1
  (0, 1107)	5
  (0, 4584)	5
  (0, 2808)	2
  (0, 1530)	1
  (0, 329)	2
  (0, 4696)	1
  (0, 2038)	1
  (0, 1354)	1
  (0, 2367)	1
  (0, 216)	1
  (0, 3987)	1
  (0, 3576)	1
  (0, 4659)	1
  (0, 2006)	1
  (0, 825)	1
  (1, 28)	3
  (1, 1256)	5
  (1, 4530)	4
  (1, 491)	5
  (1, 1461)	1
  (1, 1882)	1
  (2, 28)	2
  (2, 2614)	5
  :	:
  (690, 1519)	1
  (690, 4111)	1
  (691, 2643)	5
  (691, 2645)	1
  (691, 2646)	1
  (691, 2649)	1
  (691, 2647)	1
  (691, 2648)	1
  (691, 2497)	3
  (691, 2644)	1
  (691, 4731)	1
  (691, 2503)	2
  (691, 766)	1
  (691, 978)	1
  (691, 1958)	3
  (691, 757)	3
  (691, 286)	2
  (691, 517)	1
  (691, 47)	1
  (691, 4395)	1
  (691, 516)	1
  (691, 515)	1
  (691, 4830)	1
  (691, 4839)	1
  (691, 2508)	1


In [217]:
lr_prediction = clf.predict(X)

In [193]:
#df_us['title_channel_tags'][0]
test = df_us['title_channel_tags'][2]
cv = count_vectorizer.transform([test])
#print(test)

In [194]:
df_us['category_id'][2]

23

In [195]:
print(cv)

  (0, 180)	1
  (0, 237)	1
  (0, 256)	1
  (0, 358)	2
  (0, 412)	1
  (0, 472)	1
  (0, 566)	1
  (0, 1318)	1
  (0, 1782)	1
  (0, 1965)	1
  (0, 2204)	1
  (0, 2256)	1
  (0, 2463)	2
  (0, 2575)	2
  (0, 2576)	1
  (0, 2600)	1
  (0, 2692)	1
  (0, 2760)	4
  (0, 3029)	1
  (0, 3157)	1
  (0, 3351)	1
  (0, 3406)	1
  (0, 3412)	1
  (0, 3415)	1
  (0, 3532)	2
  (0, 3735)	4
  (0, 3736)	1
  (0, 3786)	1
  (0, 3923)	1
  (0, 3924)	1
  (0, 4221)	3
  (0, 4624)	1
  (0, 4751)	1


In [196]:
X.shape

(3457, 4933)

In [197]:
Y.shape

(3457,)

In [198]:
cv.shape

(1, 4933)

In [199]:
clf.predict(cv)[0]#.reshape(-1, 1)) #np.array

4607967.604540961

In [200]:
df_us['views'][2]

3191434

In [201]:
# print(df_us.iloc[10])

In [202]:
# clf.predict(df_us.iloc[10])

In [203]:
clf.score(X_test, y_test)

0.9166606251325035

In [204]:
df_us.head()

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,title_channel_tags
0,2kyS6SvSYSE,17.14.11,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,22,2017-11-13T17:13:01.000Z,SHANtell martin,748374,57527,2966,15954,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,False,False,False,SHANTELL'S CHANNEL - https://www.youtube.com/s...,WE WANT TO TALK ABOUT OUR MARRIAGESHANtell mar...
1,1ZAPwfrtAFY,17.14.11,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,24,2017-11-13T07:30:00.000Z,"last week tonight trump presidency|""last week ...",2418783,97185,6146,12703,https://i.ytimg.com/vi/1ZAPwfrtAFY/default.jpg,False,False,False,"One year after the presidential election, John...",The Trump Presidency: Last Week Tonight with J...
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146033,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...,"Racist Superman | Rudy Mancuso, King Bach & Le..."
3,puqaWrEC7tY,17.14.11,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,24,2017-11-13T11:00:04.000Z,"rhett and link|""gmm""|""good mythical morning""|""...",343168,10172,666,2146,https://i.ytimg.com/vi/puqaWrEC7tY/default.jpg,False,False,False,Today we find out if Link is a Nickelback amat...,Nickelback Lyrics: Real or Fake?rhett and link...
4,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12T18:01:41.000Z,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095731,132235,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...,"I Dare You: GOING BALD!?ryan|""higa""|""higatv""|""..."


In [78]:
lr_results = np.array(list(zip(df_us['title'][df_us['category_id'] == 23],lr_prediction, df_us['views'][df_us['category_id'] == 23])))

In [79]:
print(lr_results)

[['Racist Superman | Rudy Mancuso, King Bach & Lele Pons'
  '2072449.3402608763' '3191434']
 ['One Change That Would Make Pacific Rim a Classic' '1500568.0306935294'
  '295639']
 ["Using Other People's Showers" '226298.3973268498' '33980']
 ...
 ['Big Shaq Beefs Ed Sheeran About The Christmas Number 1 Spot'
  '437901.379900385' '265168']
 ['HOSTILES ON THE HILL — A Bad Lip Reading of The Empire Strikes Back'
  '8766277.619042702' '969903']
 ["I Built a 'Scream Box' So My Girlfriend Won't Go Crazy"
  '734020.2232655238' '304682']]


# Sentiment Analysis

# United States

In [58]:
sentiment_analyzer = SIA()

In [61]:
us_sentiment_title_results = []
us_sentiment_tags_results = []
us_sentiment_description_results = []

In [71]:
for i in df_us['title']:
    ps = sentiment_analyzer.polarity_scores(i)
    ps['title'] = i
    us_sentiment_title_results.append(ps)

In [72]:
for i in df_us['tags']:
    ps = sentiment_analyzer.polarity_scores(i)
    ps['tags'] = i
    us_sentiment_tags_results.append(ps)

In [78]:
#for i in df_us['description']:
#    ps = sentiment_analyzer.polarity_scores(i)
#    ps['title'] = i
#    us_sentiment_description_results.append(ps)

In [73]:
df_sentiment_us_title = pd.DataFrame.from_records(us_sentiment_title_results)
df_sentiment_us_tags = pd.DataFrame.from_records(us_sentiment_tags_results)

In [53]:
df_sentiment_us_title.head()

,neg,neu,pos,compound,title
0,0.000,0.822,0.178,0.0772,WE WANT TO TALK ABOUT OUR MARRIAGE
1,0.000,1.000,0.000,0.0000,The Trump Presidency: Last Week Tonight with J...
2,0.364,0.636,0.000,-0.6124,"Racist Superman | Rudy Mancuso, King Bach & Le..."
3,0.437,0.563,0.000,-0.4767,Nickelback Lyrics: Real or Fake?
4,0.000,1.000,0.000,0.0000,I Dare You: GOING BALD!?


In [74]:
df_sentiment_us_title['Sentiment'] = 'Neutral'
df_sentiment_us_tags['Sentiment'] = 'Neutral'

In [75]:
df_sentiment_us_title.loc[df_sentiment_us_title['compound'] > 0.4, 'Sentiment'] = 'Positive'
df_sentiment_us_title.loc[df_sentiment_us_title['compound'] < -0.4, 'Sentiment'] = 'Negative'

df_sentiment_us_tags.loc[df_sentiment_us_tags['compound'] > 0.4, 'Sentiment'] = 'Positive'
df_sentiment_us_tags.loc[df_sentiment_us_tags['compound'] < -0.4, 'Sentiment'] = 'Negative'

In [76]:
df_sentiment_us_title.head()

,neg,neu,pos,compound,title,Sentiment
0,0.000,0.822,0.178,0.0772,WE WANT TO TALK ABOUT OUR MARRIAGE,Neutral
1,0.000,1.000,0.000,0.0000,The Trump Presidency: Last Week Tonight with J...,Neutral
2,0.364,0.636,0.000,-0.6124,"Racist Superman | Rudy Mancuso, King Bach & Le...",Negative
3,0.437,0.563,0.000,-0.4767,Nickelback Lyrics: Real or Fake?,Negative
4,0.000,1.000,0.000,0.0000,I Dare You: GOING BALD!?,Neutral


In [77]:
df_sentiment_us_tags.head()

,neg,neu,pos,compound,tags,Sentiment
0,0.0,1.000,0.000,0.0000,SHANtell martin,Neutral
1,0.0,1.000,0.000,0.0000,"last week tonight trump presidency|""last week ...",Neutral
2,0.2,0.800,0.000,-0.6124,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",Negative
3,0.0,0.919,0.081,0.4404,"rhett and link|""gmm""|""good mythical morning""|""...",Positive
4,0.0,1.000,0.000,0.0000,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",Neutral


# Canada

In [79]:
ca_sentiment_title_results = []
ca_sentiment_tags_results = []
ca_sentiment_description_results = []

In [80]:
for i in df_ca['title']:
    ps = sentiment_analyzer.polarity_scores(i)
    ps['title'] = i
    ca_sentiment_title_results.append(ps)

In [81]:
for i in df_ca['tags']:
    ps = sentiment_analyzer.polarity_scores(i)
    ps['tags'] = i
    ca_sentiment_tags_results.append(ps)

In [82]:
df_sentiment_ca_title = pd.DataFrame.from_records(ca_sentiment_title_results)
df_sentiment_ca_tags = pd.DataFrame.from_records(ca_sentiment_tags_results)

In [83]:
df_sentiment_ca_title['Sentiment'] = 'Neutral'
df_sentiment_ca_tags['Sentiment'] = 'Neutral'

In [84]:
df_sentiment_ca_title.loc[df_sentiment_ca_title['compound'] > 0.4, 'Sentiment'] = 'Positive'
df_sentiment_ca_title.loc[df_sentiment_ca_title['compound'] < -0.4, 'Sentiment'] = 'Negative'

df_sentiment_ca_tags.loc[df_sentiment_ca_tags['compound'] > 0.4, 'Sentiment'] = 'Positive'
df_sentiment_ca_tags.loc[df_sentiment_ca_tags['compound'] < -0.4, 'Sentiment'] = 'Negative'

# Great Britain

In [85]:
uk_sentiment_title_results = []
uk_sentiment_tags_results = []
uk_sentiment_description_results = []

In [87]:
for i in df_uk['title']:
    ps = sentiment_analyzer.polarity_scores(i)
    ps['title'] = i
    uk_sentiment_title_results.append(ps)

In [88]:
for i in df_uk['tags']:
    ps = sentiment_analyzer.polarity_scores(i)
    ps['tags'] = i
    uk_sentiment_tags_results.append(ps)

In [89]:
df_sentiment_uk_title = pd.DataFrame.from_records(ca_sentiment_title_results)
df_sentiment_uk_tags = pd.DataFrame.from_records(ca_sentiment_tags_results)

In [90]:
df_sentiment_uk_title['Sentiment'] = 'Neutral'
df_sentiment_uk_tags['Sentiment'] = 'Neutral'

In [91]:
df_sentiment_uk_title.loc[df_sentiment_uk_title['compound'] > 0.4, 'Sentiment'] = 'Positive'
df_sentiment_uk_title.loc[df_sentiment_uk_title['compound'] < -0.4, 'Sentiment'] = 'Negative'

df_sentiment_uk_tags.loc[df_sentiment_uk_tags['compound'] > 0.4, 'Sentiment'] = 'Positive'
df_sentiment_uk_tags.loc[df_sentiment_uk_tags['compound'] < -0.4, 'Sentiment'] = 'Negative'